# Identity Management in LUSID
In this part of the course, we will describe the processes of:
- Setting up personal accounts in LUSID.
- Setting up service accounts in LUSID.
- Getting and using your access tokens.

## Setting up personal accounts in LUSID
To set up a personal account in LUSID, navigate to the users page:
- Sign into LUSID.
- Click on the hamburger in the top left corner of LUSID. A context menu should appear.
- Navigate to the "Identity and Access" menu item. A sub-menu should appear.
- Click on Users.

This page contains a table with all information about currently registered user accounts.

To create a new personal account:
- Click the "create user" button. A dialog box will appear.
- Fill out the "First name", "Last name" and "Email address" fields.
- You can optionally assign roles to users. Roles are explored in more detail later in this course.

Upon creation of the account, the user will be sent an email with a link which will instruct them on how to activiate their account, and provide credentials to sign into LUSID.




## Setting up service accounts in LUSID.
To set up a service account, first navigate to the Users page as we did in the previous section, then:
- Click the "create user" button. A dialog box will appear.
- Select the "Service" option in the "Account type" select box.
- Fill out the "Login" field. Note, this login must look like an email address, even though no emails are currently sent this address.
- Fill out the "Email address" and optionally the "Secondary email address" fields.
- You can also add roles to service accounts.

## Getting and using your access tokens.